In [ ]:
from detection_helpers import *
from tracking_helpers import *
from bridge_wrapper import *
from PIL import Image

# detect ONLY [person,horses,sports ball]. class = None means detect all classes. List info at: "data/coco.yaml"
detector = Detector(classes = [0, 17, 32])
detector.load_model('yolov7x.pt')

# Pass in any image path or Numpy Image using 'BGR' format
# plot_bb = False output the predictions as [x,y,w,h, confidence, class]
result = detector.detect('./IO_data/input/images/horses.jpg', plot_bb=True)

# If it is image, convert it to proper image. Detector will give "BGR" image
if len(result.shape) == 3:
    result = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)) 
    
result

In [4]:
!streamlit run consumer.py

^C


In [1]:
from detection_helpers import *
from tracking_helpers import *
from bridge_wrapper import *
from PIL import Image

# detect ONLY [person,horses,sports ball]
detector = Detector(classes=[2, 5, 6, 7])
detector.load_model('yolov7x.pt')

tracker = YOLOv7_DeepSORT(reID_model_path="./deep_sort/model_weights/mars-small128.pb", detector=detector)
tracker.track_video("./IO_data/input/video/test_video.mp4",
                    output="./IO_data/output/test_video.avi",
                    show_live=True,
                    skip_frames=0,
                    count_objects=True,
                    verbose=1)

Fusing layers... 
 Convert model to Traced-model... 


c:\Users\dinhc\anaconda3\envs\speed\lib\site-packages\torch\nn\modules\module.py:675: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:485.)
  if param.grad is not None:


 traced_script_module saved! 
 model is traced! 

Loading detected frame from file...
Processed frame no: 1 || Current FPS: 2.0 || Objects tracked: 12
Loading detected frame from file...
Processed frame no: 2 || Current FPS: 5.0 || Objects tracked: 12
Loading detected frame from file...
Track id: 1
Time start: [3, 0, 0, 0]
Time end: [0, 0, 0, 0]
Track id: 2
Time start: [0, 3, 0, 0]
Time end: [0, 0, 0, 0]
Track id: 3
Time start: [3, 0, 0, 0]
Time end: [0, 0, 0, 0]
Track id: 4
Time start: [3, 0, 0, 0]
Time end: [0, 0, 0, 0]
Track id: 5
Time start: [0, 0, 3, 0]
Time end: [3, 0, 0, 0]


ZeroDivisionError: division by zero

In [2]:
import pandas as pd

df=pd.read_csv('labels_1.txt', header=None)
df.head()

,0,1,2,3,4,5,6
0,0,truck,0,1045,318,88,124
1,0,truck,1,912,302,81,114
2,0,car,0,759,297,64,63
3,0,car,1,1005,286,54,53
4,2,truck,0,1045,320,89,125


In [6]:
df_train = df.loc[df[0] < 23960].reset_index(drop=True)
df_dev = df.loc[(23960 <= df[0]) & (df[0] < 25289)].reset_index(drop=True)
df_test = df.loc[df[0] >= 25289].reset_index(drop=True)

In [33]:
print(len(df_train[0].unique()))
print(len(df_dev[0].unique()))
print(len(df_test[0].unique()))

11162
595
659


In [21]:
df_train[[1,2]].drop_duplicates().groupby(1).count()

,2
1,
bus,2
car,324
truck,80
van,26


In [22]:
df_dev[[1,2]].drop_duplicates().groupby(1).count()

,2
1,
car,18
truck,7


In [23]:
df_test[[1,2]].drop_duplicates().groupby(1).count()

,2
1,
car,17
truck,14


In [ ]:
import pandas as pd

pd.DataFrame(['Train'])

In [ ]:
pd_train['set'] = 'Train'
pd_dev['set'] = 'Dev'
pd_test['set'] = 'Test'
df_pd = pd.concat([pd_train, pd_dev, pd_test])
df_pd['counter'] = 1
data = df_pd.groupby(['label_id', 'set']).sum().reset_index().replace({0: 'CLEAN', 1: 'OFFENSIVE', 2: 'HATE'})

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"

figure(figsize=(8, 6), dpi=100)
plt.style.use('ggplot')

g = sns.barplot(x = "set", y = "counter", hue='label_id', data = data, order = ["Train", "Dev", "Test"])

g.set_title('Số lượng nhãn trên mỗi tập dữ liệu (Official)', color='black')
g.set_ylabel("Số lượng nhãn", color='black')
g.set_xlabel("Tập dữ liệu", color='black')
h, l = g.get_legend_handles_labels()
label = ['CLEAN', 'OFFENSIVE', 'HATE']
g.legend(h, label, title="Nhãn")
g.set(ylim = (0, 22000))
ylabels = ['{:,.0f}'.format(x) + 'K' for x in g.get_yticks()/1000]
g.set_yticklabels(ylabels)
for container in g.containers:
    g.bar_label(container, fontsize=10, color='black')
g.set_xticklabels(['Huấn luyện', 'Phát triển', 'Kiểm thử'])
for tick in g.yaxis.get_major_ticks():
    tick.label1.set_color('black')
for tick in g.xaxis.get_major_ticks():
    tick.label1.set_color('black')
plt.show()